# Trabalho Final de Algoritmos Genéticos

___

<font size=3>Olá,</font> 

<font size=3>Este arquivo armazena as informações sobre o trabalho semestral desenvolvido na disciplina de **redes neurais e algoritmos genéticos por alunos da Ilum Escola de Ciência.**</font>

___

**Alunos:** Cauê Gomes Correia dos Santos, Izaque Junior Oliveira Silva e Karla Rovedo Pascoalini.

**Professor:** Daniel Roberto Cassar
___

<font size=3>O intuito deste trabalho é desenvolver um `Algoritmo Genético` que otimize a função `minimize_energy`, que busca a menor energia de uma molécula, de acordo com o método de Huckel, dado o número de átomos que a compõem, calculando todas as energias possíveis a partir de todas as combinações possíveis dado o número de átomos. A forma como essa função calcula a menor energia tende a ser custusa computacionalmente, como se pode imaginar. Devido a esse problema, uma otmização é necessária para que o custo computacional seja menor e possamos obter a menor energia da molécula almejada.</font>

### 1 - Importando as bibliotecas
___ 

In [1]:
import numpy as np 
import networkx as nx
from pyvis import network as net
from otimizando_menor_energia import create_upper_triangular_binary_matrix as create_traingular_umatrix

## 2 - How many atoms does your molecule have?
___

<font size=3>A função `draw_graph` foi desenvolvida para plotar o grafo da molécula de menor energia que resultou da função `minimize_energy` dado um conjunto de átomos.</font>

In [2]:
def draw_graph(networkx_graph,notebook=True,output_filename='graph.html',show_buttons=True,only_physics_buttons=True):
    """
    This function accepts a networkx graph object,
    converts it to a pyvis network object preserving its node and edge attributes,
    and both returns and saves a dynamic network visualization.
    
    Args:
        networkx_graph: The graph to convert and display
        notebook: Display in Jupyter?
        output_filename: Where to save the converted network
        show_buttons: Show buttons in saved version of network?
        only_physics_buttons: Show only buttons controlling physics of network?
    """
    
    # make a pyvis network
    pyvis_graph = net.Network(notebook=notebook)
    
    # for each node and its attributes in the networkx graph
    for node,node_attrs in networkx_graph.nodes(data=True):
        pyvis_graph.add_node(str(node),**node_attrs)
        
    # for each edge and its attributes in the networkx graph
    for source,target,edge_attrs in networkx_graph.edges(data=True):
        # if value/width not specified directly, and weight is specified, set 'value' to 'weight'
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            # place at key 'value' the weight of the edge
            edge_attrs['value']=edge_attrs['weight']
        # add the edge
        pyvis_graph.add_edge(str(source),str(target),**edge_attrs)
        
    # turn buttons on
    if show_buttons:
        if only_physics_buttons:
            pyvis_graph.show_buttons(filter_=['physics'])
        else:
            pyvis_graph.show_buttons()
    #net.repulsion(node_distance=500, spring_length=200)
    # return and also save
    return pyvis_graph.show(output_filename)



In [4]:
def Minimiza_Energia(quantidade_atomos):
    matrizes = []
    
    # Gerar todas as possíveis matrizes
    for i in range(2 ** (quantidade_atomos * (quantidade_atomos - 1) // 2)):
        H = np.zeros((quantidade_atomos, quantidade_atomos))
        bits = list(format(i, 'b').zfill(quantidade_atomos * (quantidade_atomos - 1) // 2))
        bit_index = 0
        
        # Preencher a matriz com os valores adequados
        for j in range(quantidade_atomos):
            for k in range(j + 1, quantidade_atomos):
                if bits[bit_index] == '1':
                    H[j][k] = -1
                    H[k][j] = -1
                bit_index += 1
        
        matrizes.append(H)
    
    return matrizes

def testar_matrizes(matrizes):
    t_menor = float('inf')  # Set initial smallest t_total to infinity
    melhor_matrix = None  # Initialize the matrix with the smallest t_total

    for H in matrizes:
        eigenvals = np.linalg.eigvals(H)
        autovalores = sorted(eigenvals.real)
        
        e = H.shape[0]
        
        if e % 2 == 0:
            ocupados = e // 2 
        elif e % 2 == 1:
            ocupados = e // 2 + 1
        
        t_total = 0
        for n in range(e//2):
            t_total += 2 * autovalores[n]
        if(e%2==1):
            t_total += autovalores[e//2]

        if t_total < t_menor:
            melhor_auto = autovalores
            t_menor = t_total
            melhor_matrix = H

    print(60 * '-', 'Resultado', 60 * '-')
    print(f'A matriz com o menor t_total associado é:')
    print(melhor_matrix)
    print(f'Com os autovalores {melhor_auto}')
    print(f'E com o valor associado de {t_menor}')
    return melhor_matrix

matrizes = Minimiza_Energia(6)
A = testar_matrizes(matrizes)

G=nx.from_numpy_array(A)
#nx.draw(G, with_labels=True)
#draw_graph(G)


------------------------------------------------------------ Resultado ------------------------------------------------------------
A matriz com o menor t_total associado é:
[[ 0.  0. -1. -1.  0. -1.]
 [ 0.  0.  0. -1. -1. -1.]
 [-1.  0.  0.  0. -1. -1.]
 [-1. -1.  0.  0.  0. -1.]
 [ 0. -1. -1.  0.  0. -1.]
 [-1. -1. -1. -1. -1.  0.]]
Com os autovalores [-3.4494897427831814, -0.6180339887498958, -0.6180339887498947, 1.4494897427831785, 1.6180339887498956, 1.6180339887498956]
E com o valor associado de -9.371115440565944


In [5]:
# Mutate the matrix by flipping a randomly chosen element in the upper triangular part (excluding the main diagonal)
def mutate_upper_triangular_binary(matrix):

    '''
    Muta uma matrix triangular superior, trocando um elemento 
    aleatório (fora da diagonal) de 0 para 1, ou vice-versa.

    arg: Matriz triangular superior
    return: Matriz triangular superior com uma mutação
    '''
    size = matrix.shape[0]
    
    # Escolhe um elemento aleatório (tirando a diagonal)
    i = np.random.randint(0, size - 1)
    j = np.random.randint(i + 1, size)  # Assegura que estará acima da diagonal principal
    
    # Flipa o valor do elemento escolhido
    matrix[i, j] = 1 - matrix[i, j]
    
    return matrix